<a href="https://colab.research.google.com/github/jkatz326/MouseGameV1/blob/main/CIS_4230_5230_HW1_SP_'24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to the first coding assignment for CIS 4230/5230. In this assignment, we will be going over the basics of tools used for ML as well as beginning our journey in fair ML.


Logistics:
* Due: Wednesday, February 28th, 2024
* PTS: 57

**INDIVIDUAL WORK ONLY**

Keep in mind that the Free Response Questions here are similar to ones you'll be asked on the exams, and you won't have access to ChatGPT then, so it's in your own interest to learn how to answer them yourself now.


There are a few packages we will use intimately during this course that you will want to become familiar with:

* [pandas](https://pandas.pydata.org/) - A package for dealing with datasets nicely
* [NumPy](https://numpy.org/doc/stable/index.html) - The underpinnings of every ML matrix operation. NumPy is extremely fast but less user friendly.
* [scikit-learn](https://scikit-learn.org/stable/index.html) - A user friendly package for tabular ML.
* [folktables](https://github.com/zykls/folktables) - Census dataset loader

If you have questions during the homework about any of these packages, look through the documentation before sending TA's questions. An important skill is being able to debug your own code, and as TA's, **it is NOT OUR JOB TO DEBUG IT FOR YOU, NOR WILL WE**. Conceptual questions are completely fair game, but "why isn't this working" is not. If you can't figure out how to get a certain slicing of data in NumPy, a google search will do you wonders (and is exactly what we would have to do to debug it anyway).

The only package that is not standard loaded on Google Colab is Folktables. You can pip install the package with the following command.

# Setup

In [ ]:
!pip install penngrader-client
!pip install folktables

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
import folktables as folk
from penngrader.grader import *

In [ ]:
### TODO (0 PTS) ###

# Enter your 8 digit Pennkey here:
STUDENT_ID = 12345678
SECRET = STUDENT_ID

# Enter your full name here (as a string):
name = "First Last"

In [ ]:
#Run this block
%%writefile config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

In [ ]:
# Initialize penngrader
grader = PennGrader('config.yaml', 'cis5230_sp24_HW1', STUDENT_ID, SECRET)

# Data Cleaning

From folktables, you are going to be using 2018 census data from Arizona to try and predict if an individual makes above or below $50,000 annually. This type of problem is known as 'binary classification'. The cells below will import the data into a pandas dataframe and split it into train + test data.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Import data
from sklearn.model_selection import train_test_split

data_source = folk.ACSDataSource(survey_year = '2018', horizon = '5-Year', survey = 'person')
acs_data = data_source.get_data(states = ["AZ"], download = True)
data_np, labels, _ = folk.ACSIncome.df_to_numpy(acs_data)
data = pd.DataFrame(data_np, columns = ['AGEP',
                                        'COW',
                                        'SCHL',
                                        'MAR',
                                        'OCCP',
                                        'POBP',
                                        'RELP',
                                        'WKHP',
                                        'SEX',
                                        'RAC1P'])
# Split data
x_train, x_test, y_train, y_test = train_test_split(data,
                                                    labels,
                                                    test_size = .2,
                                                    random_state = 99)

You can view the first n entries of a pandas dataframe using X.head(n)

In [ ]:
# View
x_train.head(15)

Each row is an individual, and each column represents a feature of the individual. You can see the exact data dictionary for columns here, but here's a general snapshot of what columns represent:

* 'AGEP' -> Age
* 'COW' -> Class of Worker
* 'SCHL' -> Education Level
* 'MAR' -> Marital Status
* 'OCCP' -> Occupation
* 'POBP' -> Place of Birth
* 'RELP' -> Relationship
* 'WKHP' -> Work Hours per Week
* 'SEX' -> Binary Sex
* 'RAC1P' -> Race

What about the labels? And how many instances are in our dataset? Number of features? Class balance?

In [ ]:
# View
print(f'First fifteen training labels: {y_train[0:15]}')
print(f'# of instances in x_train: {x_train.shape[0]}')
print(f'# of features in x_train: {x_train.shape[1]}')
print(f'Number of True values in training labels: {y_train.sum()}')
print(f'Number of False values in training labels: {len(y_train) - y_train.sum()}')

Looking at this type of information is generally the first thing you should examine in a dataset when performaning EDA (exploratory data analysis). Other factors you may want to consider is the type of features (categorical, ordinal, real value), range of feature values, etc.

In [ ]:
### TODO ### (2 PTS)

# get the first 15 individuals of x_train
x_train_15 = # Your Code Here

# get individuals 15 thru 30 of x_train
x_train_slice = # Your Code Here

In [ ]:
# View
print(x_train_15)

In [ ]:
# View
print(x_train_slice)

In [ ]:
### DONT CHANGE THIS CELL
grader.grade(test_case_id = 'check_train', answer = (x_train_15.to_numpy(), x_train_slice.to_numpy()))

Instead of getting rows, you may (will) need to get columns.

In [ ]:
### TODO ### (2 Pts)

# select the first 15 individuals in x_train age column
x_train_age = # Your Code Here


# select the first 15 individuals in x_train education column
x_train_school = # Your Code Here


In [ ]:
x_train_age

In [ ]:
x_train_school

In [ ]:
grader.grade(test_case_id = 'check_train_age_school', answer = (np.array(x_train_age), np.array(x_train_school)))

In [ ]:
# View
print(x_train_age)
print(type(x_train_age))
print(x_train_age.to_numpy())

In [ ]:
# View
print(x_train_school)
print(type(x_train_school))
print(x_train_school.to_numpy())

While it is fun to get whole columns or rows, it is much more useful to get rows based on some value in a specified column. For example,what if we want to get all male or female individuals in this dataset?

The same function that let you look at columns will let you create a boolean True/False indicator function by specifying X.column_name == value or X['column_name] == value

In [ ]:
# Example
print(x_train.SEX == 1)

Create functions that define the following four groups:

individuals identified as white
individuals identified as black
individuals identified as male
individuals identified as female
You will use these functions later on in the notebook.

In [ ]:
#Todo (8 PTS)

def g_white(X):
  '''Indicator function which returns boolean True/False
   for individuals identified as white'''
  indices = # Your Code Here

  return indices

def g_black(X):
    '''Indicator function which returns boolean True/False
     for individuals identified as black'''
    indices = # Your Code Here

    return indices


def g_male(X):
    '''Indicator function which returns boolean True/False
     for individuals identified as male'''
    indices = # Your Code Here

    return indices


def g_female(X):
    '''Indicator function which returns boolean True/False
     for individuals identified as female'''
    indices = # Your Code Here

    return indices

In [ ]:
# Don't modify this cell but run it :)
white_indices = g_white(x_train_15)
black_indices = g_black(x_train_15)
male_indices = g_male(x_train_15)
female_indices = g_female(x_train_15)

In [ ]:
# Don't modify this cell but run it :)
white_indices_num = g_white(x_train).sum()
black_indices_num = g_black(x_train).sum()
male_indices_num = g_male(x_train).sum()
female_indices_num = g_female(x_train).sum()

In [ ]:
grader.grade(test_case_id = 'check_white_indices', answer = (white_indices.to_numpy(), white_indices_num))
grader.grade(test_case_id = 'check_black_indices', answer = (black_indices.to_numpy(), black_indices_num))
grader.grade(test_case_id = 'check_male_indices', answer = (male_indices.to_numpy(), male_indices_num))
grader.grade(test_case_id = 'check_female_indices', answer = (female_indices.to_numpy(), female_indices_num))

Your group indicator functions return boolean True/False values, but you may want the slice from the dataframe defined by indicator function. Pandas lets you slice data using indicator functions by doing X[True/False series].



In [ ]:
### TODO ### (3 PTS)

# write a function which accepts a dataframe X and group g and returns the subset of individuals who are a part of group g
def group_data(X, g):
    # Your Code Here

    return X_subset

In [ ]:
# Don't modify this cell but run it :)
data = group_data(x_train[0:15], g_white)

In [ ]:
grader.grade(test_case_id = 'check_group_data', answer = data.to_numpy())

Given the feature space of a dataset, the number of groups you can define grows exponentially. If you have  𝑑  features which can take on 2 values each, then

**|unique groups| =  $2^𝑑$**

However, in many datasets such as this one, the number of values each feature can take on is larger than 2. The cell below will show the number of groups that can be made given that feature values are exactly some value (i.e. not including a group where age < 40, but includes age = 40).

In [ ]:
# Number of unique groups
print(f'Number of unique groups: {np.prod(x_train.nunique())}')

Number of unique groups: 268070803153920


Clearly there are a lot of unique groups in this dataset, going down to the individual defining their own group (unless there are two individuals with identical feature values in the dataset). The groups you defined above were quite large, check the sizes in the training data for each function using your earlier group functions. You will need to determine how to get the number of True values.

In [ ]:
### TODO ### (4 Pts)

# find number of individuals identified as white in x_train

size_train_white = 0
# Your Code Here



# find number of individuals identified as black in x_train
size_train_black = 0
# Your Code Here



# find number of individuals identified as male in x_train
size_train_male = 0
# Your Code Here


# find number of individuals identified as female in x_train
size_train_female = 0
# Your Code Here



In [ ]:
# View
print("# of white individuals = " + str(size_train_white))
print("# of black individuals = " + str(size_train_black))
print("# of men = " + str(size_train_male))
print("# of women = "  + str(size_train_female))

In [ ]:
grader.grade(test_case_id = 'check_size_train_white', answer = size_train_white)
grader.grade(test_case_id = 'check_size_train_black', answer = size_train_black)
grader.grade(test_case_id = 'check_size_train_male', answer = size_train_male)
grader.grade(test_case_id = 'check_size_train_female', answer = size_train_female)

**FRQ 1:**

What do you notice about the number of individuals that fall within each category? How can this effect the model's performance on specific groups of people? Answer in 1-2 Paragraphs. (5 PTS)

**Answer:**




It is worth noting you can immediately make more groups using logical AND/OR to create intersection and unions of the boolean indicator functions.

In [ ]:
### TODO ### (2 PTS)

# get boolean indices of individuals who are indentified as white OR black in x_train_15
indices_black_or_white = # Your Code Here


# get boolean indices of individuals who are indentified as white AND male in x_train_15
indices_white_and_male = # Your Code Here


In [ ]:
#Run but do not modify
grader.grade(test_case_id = 'check_black_white', answer = indices_black_or_white.to_numpy())
grader.grade(test_case_id = 'check_white_male', answer = indices_white_and_male.to_numpy())

We've gone over the basics of Pandas dataframes to look at/slice dataframes, but how do we create preditive models using this data? Well, we want to minimize an objective function over a hypothesis class using an optimization method while having good generalization. Training a decision tree or a neural network both follow this procedure, but one is much more complicated. This course is not designed to show you methods which complete this optimization task (CIS 5190/5200 would be better if you are looking for that), but rather to see/measure/fix the shortcomings of this method.

# Model Training

Time to train some models.

Below we give the generic pipeline for training a ML model on tabular dataset:

In [ ]:
# import decision tree classifier
from sklearn.tree import DecisionTreeClassifier

# import loss function
from sklearn.metrics import zero_one_loss as loss_fn

The loss function we will be using is the zero-one loss, also known as hit loss. It is defined as
* $\frac{1}{n}\sum_{i = 1}^{n} I(\hat{y} \neq y)$

where $𝑦$ is the true value, $𝑦̂$  is the predicted value, $𝑛$ is the number of samples, and 𝐼 is indicator function.

More simplistically, hit loss is the percentage of time your model makes the incorrect prediction.

Here is an example of training a depth 3 decision tree. Note that we fit our model on the training samples, and then predict on both the train and test data. Viewing both errors will give you a sense of how your model is generalizing, and is usually done with a validation vice a test set since you gain information about the test set.

In [ ]:
### Generic ML Pipeline ###

# Define hypothesis class: decision trees with max depth 3
clf = DecisionTreeClassifier(max_depth = 3, random_state = 42)

# Minimize zero one loss using training data
clf.fit(x_train, y_train)

# Get train/test predictions from trained hypothesis
train_predictions = clf.predict(x_train)
test_predictions = clf.predict(x_test)

# Compute train/test error
train_error = loss_fn(y_train, train_predictions)
test_error = loss_fn(y_test, test_predictions)

# Print errors
print(f'Train Error: {train_error:.2%}')
print(f'Test Error: {test_error:.2%}')

Both train and test errors are roughly equivalent, though pretty high. What happens when you train a deeper decision tree, say of depth 12?



In [ ]:
### TODO ### (3 PTS)

### MUST SET RANDOM_STATE = 99 ###

# define hypothesis class of max depth 12 decision trees decision, fit the model, and record train and test errors
# Your Code Here


# print errors
print(f'Train Error: {dt_12_train_error:.2%}')
print(f'Test Error: {dt_12_test_error:.2%}')

In [ ]:
#Run but do not modify
grader.grade(test_case_id = 'check_dt_12_train_test_error', answer = (dt_12_train_error, dt_12_test_error))

Notice both train and test error have decreased, but a bit of separation between the two has occurred. Now what happens in an extremely complex hypothesis space?

In [ ]:
### TODO ### (3 Pts)

### MUST SET RANDOM_STATE = 99 ###

# define hypothesis class of max depth 29 decision trees decision, and record train and test errors
# Your Code Here

# print errors
print(f'Train Error: {dt_29_train_error:.2%}')
print(f'Test Error: {dt_29_test_error:.2%}')

In [ ]:
#Run but do not modify
grader.grade(test_case_id = 'check_dt_29_train_test_error', answer = (dt_29_train_error, dt_29_test_error))

**FRQ 2:**

Now there is a drastic difference between the two!
Define model overfitting, and analyze whether or not this model is overfit to the training data. Answer in 2-3 sentences. (2 PTS).

**Answer:**



In [ ]:
!pip install seaborn

In [ ]:
# About 30 seconds to run

import matplotlib.pyplot as plt
import seaborn as sns

train_error_list = []
test_error_list = []

def test_train_graph(errors_dict, title='Blank', x_axis = 'Rounds', y_axis = 'Error'):
    random_set = list(errors_dict.values())[0]
    x = np.linspace(1, len(random_set), len(random_set))
    errors_dict['axis'] = x
    graph_df = pd.DataFrame.from_dict(errors_dict)

    graph_df_melted = graph_df.melt('axis',var_name = 'Errors Set', value_name = 'Error')

    ### SET GRAPHS LABELS HERE ###
    xaxis =  x_axis
    yaxis =  y_axis

    plt.figure(figsize = (15,10))
    sns.set_style("whitegrid")
    sns_plt = sns.lineplot(data=graph_df_melted, x='axis', y='Error', hue='Errors Set')
    sns_plt.set_title(title, fontsize = 15)
    sns_plt.set_xlabel(xaxis, fontsize = 13)
    sns_plt.set_ylabel(yaxis, fontsize = 13)
    sns_plt.legend(bbox_to_anchor=(.98, 1))
    plt.show(block=False)

for i in range(1, 30):
    clf = DecisionTreeClassifier(max_depth = i, random_state = 42)
    clf.fit(x_train, y_train)

    train_predictions = clf.predict(x_train)
    test_predictions = clf.predict(x_test)

    train_error = loss_fn(y_train, train_predictions)
    test_error = loss_fn(y_test, test_predictions)

    train_error_list.append(train_error)
    test_error_list.append(test_error)

test_train_graph({"Train Error":train_error_list, "Test Error":test_error_list}, title = 'Error on Decision Trees as Max Depth Increases', x_axis = 'Depth', y_axis = 'Loss')

In [ ]:
### TODO ### (3 Points)

# put integer depth of underfitting (answer is a range, no exact answer expected)
underfitting_depth = # Your Code Here


# put integer depth good generalization (answer is a range, no exact answer expected)
good_generalization_depth = # Your Code Here


# put integer depth of overfitting (answer is a range, no exact answer expected)
overfitting_depth = # Your Code Here


In [ ]:
#Run but do not modify
grader.grade(test_case_id = 'check_fitting_depth', answer = (underfitting_depth, good_generalization_depth, overfitting_depth))

Using the framework above, train a decision tree classifier which has the best out of sample performance.

In [ ]:
### TODO ### (1 Point)

### MUST SET RANDOM_STATE = 99 ###
### MUST SET MAX DEPTH TO 10 ###

# train a decision tree classifier
# Your Code Here

# We will manually grade this problem

# Model Evaluation

Great, now that you've created a model, it's time to evaluate how it does on different metrics.

Write two functions that calculate the following:
* False Postive Rate = $\frac{FP}{FP + TN}$
* False Negative Rate = $\frac{FN}{FN + TP}$

where FP is false positive, FN is false negative, TP is true positive, and TN is true negative.

A useful function you may want to look at is sklearn's confusion matrix...

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
### TODO ### (0 Pts) *We will test this in the next section*

# write function which returns false positive rate given true y and predicted y
def FPR(y_true, y_pred):
    # Your Code Here
    return fpr

# write function which returns false negative rate given true y and predicted y
def FNR(y_true, y_pred):
    # Your Code Here

    return fnr

In [ ]:
### TODO ### (7 Pts)

# compute false positive rate on dt_clf (A range of answers will be accepted)
fpr_test = # Your Code Here


# compute false negative rate on dt_clf (A range of answers will be accepted)
fnr_test = # Your Code Here


print(f'Test false positive rate: {fpr_test}')
print(f'Test false negative rate: {fnr_test}')

In [ ]:
grader.grade(test_case_id = 'check_fpr_test', answer = (fpr_test))
grader.grade(test_case_id = 'check_fnr_test', answer = (fnr_test))

Is this a high false positive rate? Low? High/low false negative rate? Is this a bad thing?

This questions often times depend entirely on the underlying problem. Suppose we are in a medical setting where a ML model is making predictions on whether an image of a tumor determines benign (0) or cancerous (1).

A high false positive rate implies that your model ofetn says an image is cancerous when it is in fact not. This isn't so bad; the image gets flagged as cancerous and then gets sent to the doctor who says the computer lied and the patient is relieved.

A high false negative rate in this case is much more dangerous: an image often times passes through as non-cancerous when it is in fact cancerous. If the doctor only looks at images which are flagged as cancerous, this model places the person in danger.

In the census data setting, you can imagine this is a consumer lending model where if someone makes more than $50,000 annually, they will get a loan. But knowing an overall model has high/low FPR/FNR only gives you a general idea of places the model can go wrong. But you can measure well-defined groups FPR/FNR as well.



In [ ]:
# Make FPR calculations possible (Run this block without modifying anything)

white_test = group_data(x_test, g_white)
white_true = y_test[g_white(x_test)]
white_pred = dt_best_clf.predict(white_test)
black_true = y_test[g_black(x_test)]
print(black_true.size)
print(white_true.size)
black_test = group_data(x_test, g_black)
black_pred = dt_best_clf.predict(black_test)
male_true = y_test[g_male(x_test)]
male_test = group_data(x_test, g_male)
male_pred = dt_best_clf.predict(male_test)
female_true = y_test[g_female(x_test)]
female_test = group_data(x_test, g_female)
female_pred = dt_best_clf.predict(female_test)

In [ ]:
### TODO ### (4 PTS)

# Compute the FPR for individuals identified as white on test data
fpr_white = # Your Code Here


# Compute the FPR for individuals identified as black on test data
fpr_black = # Your Code Here


# Compute the FPR for individuals identified as male on test data
fpr_male = # Your Code Here

# Compute the FPR for individuals identified as female on test data
fpr_female = # Your Code Here


print(f'FPR individuals (white): {fpr_white}')
print(f'FPR individuals (black): {fpr_black}')
print(f'FPR individuals (male): {fpr_male}')
print(f'FPR individuals (female): {fpr_female}')

In [ ]:
grader.grade(test_case_id = 'check_fpr_white', answer = (fpr_white))
grader.grade(test_case_id = 'check_fpr_black', answer = (fpr_black))
grader.grade(test_case_id = 'check_fpr_male', answer = (fpr_male))
grader.grade(test_case_id = 'check_fpr_female', answer = (fpr_female))

Note that the only group drastically far away from the model's overall FPR is individuals identified as females, which is significantly lower.



What about FNR?

In [ ]:
### TODO ### (4 PTS)

# Compute the FNR for individuals identified as white on test data
fnr_white = # Your Code Here


# Compute the FNR for individuals identified as black on test data
fnr_black = # Your Code Here


# Compute the FNR for individuals identified as male on test data
fnr_male = # Your Code Here

# Compute the FNR for individuals identified as female on test data
fnr_female = # Your Code Here


print(f'FNR individuals (white): {fnr_white}')
print(f'FNR individuals (black): {fnr_black}')
print(f'FNR individuals (male): {fnr_male}')
print(f'FNR female individuals (female): {fnr_female}')

In [ ]:
grader.grade(test_case_id = 'check_fnr_white', answer = (fnr_white))
grader.grade(test_case_id = 'check_fnr_black', answer = (fnr_black))
grader.grade(test_case_id = 'check_fnr_male', answer = (fnr_male))
grader.grade(test_case_id = 'check_fnr_female', answer = (fnr_female))

Looking at the FPR and FNR metrics on different groups is one way of quickly checking for bias. But let's plot out the AUC-ROC curves as described in Northpointe's response to ProPublica.

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
# Run this block without modifying anything
y_score_male = dt_best_clf.predict_proba(male_test)[:, 1]

FPR_MALE, TPR_MALE, _ = roc_curve(male_true, y_score_male)
roc_auc = auc(FPR_MALE, TPR_MALE)

plt.figure()
plt.plot(FPR_MALE, TPR_MALE, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for male subgroup')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Run this block without modifying anything
y_score_female = dt_best_clf.predict_proba(female_test)[:, 1]

FPR_FEMALE, TPR_FEMALE, _ = roc_curve(female_true, y_score_female)
roc_auc = auc(FPR_FEMALE, TPR_FEMALE)

plt.figure()
plt.plot(FPR_FEMALE, TPR_FEMALE, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for female subgroup')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Run this block without modifying anything
y_score_white = dt_best_clf.predict_proba(white_test)[:, 1]

FPR_WHITE, TPR_WHITE, _ = roc_curve(white_true, y_score_white)
roc_auc = auc(FPR_WHITE, TPR_WHITE)

plt.figure()
plt.plot(FPR_WHITE, TPR_WHITE, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for white subgroup')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Run this block without modifying anything
y_score_black = dt_best_clf.predict_proba(black_test)[:, 1]

FPR_BLACK, TPR_BLACK, _ = roc_curve(black_true, y_score_black)
roc_auc = auc(FPR_BLACK, TPR_BLACK)

plt.figure()
plt.plot(FPR_BLACK, TPR_BLACK, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for black subgroup')
plt.legend(loc="lower right")
plt.show()

**FRQ 3:**

Why might black individuals have a significantly higher FNR than white individuals in this dataset? Answer in 2-3 sentences. (3 PTS)

**Answer:**

*Write your answer here*

**FRQ 4:**

Describe the effect abnormally high FNR in the context of a loan application. Answer in 2-3 sentences. (3 PTS)

**Answer:**

*Write your answer here*

**FRQ 5:**

What do you notice about the ROC curve and AUC for each of the different subgroups? What does that say about the bias in the model? Answer in 2-3 sentences. (3 PTS)

**Answer:**

Submission Instructions:
Stay tuned for Gradescope submission instructions!
